# Structual Diversification surrounding a Scaffold 
## for PF-06446846 (highly selective inhibitor of translation of PCSK9)

This code will take in the SMILES of a molecule and build off the molecule where possible using reactions that have been supplied using reaction SMIRKS (Reductive Aminations,...). Then, the code will create overlays of 3D conformers of the final molecules and generate clusters. Then, it can count building blocks as they appear both within and between clusters. 

In [ ]:
# Import OpenEye stuff
import openeye.oechem as oechem
import openeye.oedepict as oedepict
import openeye.oemolprop as oemolprop
from IPython.display import display
import openeye.oeomega as oeomega
import openeye.oeshape as oeshape
from openeye.oeshape import *



# Add utility function for depiction
def depict(mol, width=500, height=200):
    from IPython.display import Image
    dopt = oedepict.OEPrepareDepictionOptions()
    dopt.SetDepictOrientation( oedepict.OEDepictOrientation_Horizontal)
    oedepict.OEPrepareDepiction(mol, dopt)
    opts = oedepict.OE2DMolDisplayOptions(width, height, oedepict.OEScale_AutoScale)
    disp = oedepict.OE2DMolDisplay(mol, opts)
    ofs = oechem.oeosstream()
    oedepict.OERenderMolecule(ofs, 'png', disp)
    ofs.flush()
    return Image(data = "".join(ofs.str()))

## REDUCTIVE AMINATIONS
1. Read in reactants (aldehydes and primary amines) 
 
2. Set up reaction conditions using SMIRKS. 

    The SMIRSK used for the amine here is set up for a secondary amine, but a primary amine's SMIRK is: " [C:1](=[O:2]).[C:3][N:4][C:7]>>[C:1][N:4][C:7]".

    The SMIRKS to react the scaffold as an aldehyde should be paired by entering the scaffold as position 0 in a primary amine reaction.



In [ ]:
#initialize stream to read in molecules and create a list storing all the molecules
# We also want to attach the SD data containing the "reactants" or building blocks that led to each product 
# this will allow us to trace back products in each cluster to its beginnings. Incoming molecules have a filter of 200 Da except for BR/Cl containing ones.

######Reading in Aldehydes######

istream = oechem.oemolistream('Enamine_Aldehydes_5798cmpds_wSmiles.sdf')
aldehydes = []

for oemol in istream.GetOEGraphMols():
    # Create a smiles string using OpenEye OEChem
    #print(oechem.OEMolToSmiles(oemol))
    if oechem.OECalculateMolecularWeight(oemol) <= 200:
        aldehydes.append(oechem.OEMolToSmiles(oemol))
    elif ("Br" in oechem.OEMolToSmiles(oemol) or "Cl" in oechem.OEMolToSmiles(oemol)):
        aldehydes.append(oechem.OEMolToSmiles(oemol))
    
istream.close()
        
print(len(aldehydes))


#####Reading in Amines######

istream = oechem.oemolistream('Enamine_Primary_Amines_30459cmpds_wSmiles.sdf')
primary_amines = []

for oemol in istream.GetOEGraphMols():
    # Create a smiles string using OpenEye OEChem
    #print(oechem.OEMolToSmiles(oemol))
    if oechem.OECalculateMolecularWeight(oemol) <= 200:
        primary_amines.append(oechem.OEMolToSmiles(oemol))
    elif ("Br" in oechem.OEMolToSmiles(oemol) or "Cl" in oechem.OEMolToSmiles(oemol)):
        primary_amines.append(oechem.OEMolToSmiles(oemol))

istream.close()
        
print(len(primary_amines))


Scaffold as secondary amine reacting with an aldehyde


In [ ]:
#####Scaffold as secondary amine reacting with an aldehyde#####

libgen = oechem.OELibraryGen("[N:4]([#6:6])[#6:7].[C:1](=[O:2])>>[C:1][N:4]([#6:6])[#6:7]")
libgen.SetExplicitHydrogens(False)
libgen.SetValenceCorrection(True)

# name the products after the reaction so you can identify which list you want later 
SecAmine_RedAm_pdts = []

mol = oechem.OEGraphMol()
oechem.OESmilesToMol(mol, 'O=C(N(C1=NC=CC=C1Cl)[C@H]2CNCCC2)C3=CC=C(N4N=NC5=CC=CN=C54)C=C3')#Enter in the SMILES of the scaffold here 
libgen.SetStartingMaterial(mol, 0)
mol.Clear()

#Loop through all reactant aldehydes
for i, ald in enumerate(aldehydes):
    
    if i < 1500: #specify number of reactants to go through
        
        oechem.OESmilesToMol(mol, aldehydes[i])
        libgen.SetStartingMaterial(mol, 1)
        
        # loop through products and add them to a list along with reactant data and other properties
        for product in libgen.GetProducts():
            #print(oechem.OEMolToSmiles(product))
            oechem.OEAddSDData(product,"Aldehyde Reactant", aldehydes[i])
            oechem.OEAddSDData(product,"Molecular Weight", str(oechem.OECalculateMolecularWeight(product)))
            oechem.OEAddSDData(product, "Lipinski Acceptors",str(oemolprop.OEGetLipinskiAcceptorCount(product)))
            oechem.OEAddSDData(product, "Lipinski Donors",str(oemolprop.OEGetLipinskiDonorCount(product)))
            oechem.OEAddSDData(product, "Rotatable Bonds", str(oemolprop.OEGetRotatableBondCount(product)))
            try:
                oemolprop.OEGetXLogP(product)
                oechem.OEAddSDData(product, "XlogP", str(oemolprop.OEGetXLogP(product)))
            except RuntimeError:
                print("Error: product=",oechem.OEMolToSmiles(product))
                                
            oemol.SetTitle("PCSK9 + Aldehyde: %s" % (oechem.OEGetSDData(product, "Aldehyde Reactant")))
            SecAmine_RedAm_pdts.append(oechem.OEMol(product))
        
        if i % 10 == 0: #match value to idx_amine contraint value
            print(f'{i}') #count to check progress based on # aldehydes reacted
        mol.Clear()    
    
    else:
        mol.Clear()
        break
        

print('Number of Compounds:' + str(len(SecAmine_RedAm_pdts)))

for product in SecAmine_RedAm_pdts:
    print('\n'+ oechem.OEMolToSmiles(product)) 


omega = oeomega.OEOmega()
omega.SetMaxConfs(1)
omega.SetStrictStereo(False)

ofs = oechem.oemolostream('ScaffoldBuilds_SecAm_RedAm.sdf')
ofs.SetFormat(oechem.OEFormat_SDF)

for oemol in SecAmine_RedAm_pdts:
    omega(oemol)
    oechem.OEWriteMolecule(ofs, oemol)
ofs.close()

Scaffold as aldehyde reacting with primary amine

In [ ]:
libgen = oechem.OELibraryGen("[C:1](=[O:2]).[N:4]([#6:6])[#6:7]>>[C:1][N:4]([#6:6])[#6:7]")
libgen.SetExplicitHydrogens(False)
libgen.SetValenceCorrection(True)

# name the products after the reaction so you can identify which list you want later 
Ald_RedAm_pdts = []

mol = oechem.OEGraphMol()
oechem.OESmilesToMol(mol, 'O=C(N(C1=NC=CC=C1Cl)[C@H]2CNCCC2)C3=CC=C(N4N=NC5=CC=CN=C54)C=C3')#Enter in the SMILES of the scaffold here 
libgen.SetStartingMaterial(mol, 0)
mol.Clear()

#Loop through all reactant aldehydes
for i, am in enumerate(primary_amines):
    
    if i < 1500: #specify number of reactants to go through
        
        oechem.OESmilesToMol(mol, primary_amines[i])
        libgen.SetStartingMaterial(mol, 1)
        
        # loop through products and add them to a list along with reactant data and other properties
        for product in libgen.GetProducts():
            #print(oechem.OEMolToSmiles(product))
            oechem.OEAddSDData(product,"Primary Amine Reactant", primary_amines[i])
            oechem.OEAddSDData(product,"Molecular Weight", str(oechem.OECalculateMolecularWeight(product)))
            oechem.OEAddSDData(product, "Lipinski Acceptors",str(oemolprop.OEGetLipinskiAcceptorCount(product)))
            oechem.OEAddSDData(product, "Lipinski Donors",str(oemolprop.OEGetLipinskiDonorCount(product)))
            oechem.OEAddSDData(product, "Rotatable Bonds", str(oemolprop.OEGetRotatableBondCount(product)))
            try:
                oemolprop.OEGetXLogP(product)
                oechem.OEAddSDData(product, "XlogP", str(oemolprop.OEGetXLogP(product)))
            except RuntimeError:
                print("Error: product=",oechem.OEMolToSmiles(product))
                                
            oemol.SetTitle("PCSK9 + Amine: %s" % (oechem.OEGetSDData(product, "Primary Amine Reactant")))
            Ald_RedAm_pdts.append(oechem.OEMol(product))
        
        if i % 10 == 0: #match value to idx_amine contraint value
            print(f'{i}') #count to check progress based on # aldehydes reacted
        mol.Clear()    
    
    else:
        mol.Clear()
        break
        

print('Number of Compounds:' + str(len(Ald_RedAm_pdts)))

for product in Ald_RedAm_pdts:
    print('\n'+ oechem.OEMolToSmiles(product)) 


omega = oeomega.OEOmega()
omega.SetMaxConfs(1)
omega.SetStrictStereo(False)

ofs = oechem.oemolostream('ScaffoldBuilds_Ald_RedAm.sdf')
ofs.SetFormat(oechem.OEFormat_SDF)

for oemol in Ald_RedAm_pdts:
    omega(oemol)
    oechem.OEWriteMolecule(ofs, oemol)
ofs.close()

# Overlay Generation

In [ ]:
import numpy as np
import random
# Initialize OEOmega object and set the max configurations per molecule. Set strict stero and atom types to false to keep restrictions loose
#omega = oeomega.OEOmega() 
omega.SetMaxConfs(1)
omega.SetStrictStereo(False)
omega.SetStrictAtomTypes(False)

ofs = oechem.oemolostream('fitted_products.sdf')
ofs.SetFormat(oechem.OEFormat_SDF)

#initialize matrix with dimesions needed 
dim = 30
overlay_dist = np.zeros(shape=(dim,dim))
overlay_Confs = []

istream = oechem.oemolistream('ScaffoldBuilds_Ald_RedAm.sdf') # change depending on which products you want
molecules = []

for oemol in Ald_RedAm_pdts[0:dim]:# change depending on which products you want
    omega(oemol)
    molecules.append(oemol)
   
    
random.shuffle(molecules) #shuffle so they aren't in order of reactant




# Setup ROCS to provide specified number of conformers per hit
# Add our molecule as the one we are fitting    
    
for i, mol in enumerate(molecules[0:dim]):
   
    options = OEROCSOptions()
    options.SetNumBestHits(1)
    #options.SetConfsPerHit(10)
    rocs = OEROCS(options)
    rocs.AddMolecule(oechem.OEMol(mol))                             #add in reference molecule
    
    
    for j, mol2 in enumerate(molecules[0:dim]):
        if i==j:                                       #take out values on the diagonal, or all the "self" pairs
            continue
        # Loop over results and output
        #omega(mol2)
        for res in rocs.Overlay(mol2):                #add in overlay "test" molecule
            outmol = res.GetOverlayConf()              #Use GetOverlayConf to get just the best; GetOverlayConfs for all
            oeshape.OERemoveColorAtoms(outmol)
            oechem.OEAddExplicitHydrogens(outmol)
            outmol.SetTitle(oechem.OEMolToSmiles(outmol))
   
            oechem.OEAddSDData(outmol,"Tanimoto", str(res.GetTanimotoCombo()))
            oechem.OEAddSDData(outmol, "Molecule 1",  oechem.OEMolToSmiles(mol1))
            oechem.OEAddSDData(outmol, "Molecule 2",  oechem.OEMolToSmiles(mol2))
            
            overlay_Confs.append(oechem.OEMol(outmol))
            overlay_dist[i,j] = (2 - res.GetTanimotoCombo()) if res.GetTanimotoCombo() <= 2 else 0 #get "distance" from tanimoto score by reversing it; sill set values of the diagonals to 0 (same molecules are compared)
            
            print(overlay_dist[i,j])
            oechem.OEWriteMolecule(ofs, outmol)
            
            
ofs.close()
#print (overlay_dist)

# Using DBScan for clustering based on 3D structure¶
This method does not need a set number of clusters to be pre-specified. You can specify variation limits of each cluster. We will read in the reverse Tanimoto scores calculated for each conformer in the previous section as a measure of distance.

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn import metrics

# eps: Controls maximum distance between two samples to be considered as in neighborhood of the other.
# min_samples: Minimum number of samples (compounds) near a compound for it to be considered a core point
# Metric: "precomputed" means use precomputed distance matrix

clustering = DBSCAN(eps=0.2, min_samples = 3, metric="precomputed")

# Fit clustering
db = clustering.fit(overlay_dist)

# Pull labels
labels = db.labels_
#print(labels)

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print('Estimated number of clusters: %d' % n_clusters_)
print('Total number of points: %d' % len(overlay_dist))
print('Estimated number of noise points: %d' % n_noise_)



# Determine how many compounds are in each cluster.
# Cluster "-1" is the "outliers"/noise points that are not in clusters.

mols_by_cluster = {}
cluster_nrs = set(labels)

for label in cluster_nrs:
    mols_by_cluster[label] = []
    
 # Clarify what this part is doing?   
    for (idx, thislabel) in enumerate(labels):
        if thislabel == label:
            mols_by_cluster[label].append(molecules[idx])
            
    print("%d molecules in cluster %s" % (len(mols_by_cluster[label]), label))


# Out of Notebook Depiction

In [ ]:
from openeye import oedepict
import shutil
import os
if os.path.isdir('846 clusters_pdfs'): shutil.rmtree('846 clusters_pdfs')
os.mkdir('846 clusters_pdfs')

for label in mols_by_cluster:
    if label=='-1':
        continue
        
    oemols = [ oechem.OEMol(mol) for mol in mols_by_cluster[label]]
    itf = oechem.OEInterface()
    PageByPage = True
    suppress_h = True
    rows = 10
    cols = 3
    ropts = oedepict.OEReportOptions(rows, cols)
    ropts.SetHeaderHeight(25)
    ropts.SetFooterHeight(25)
    ropts.SetCellGap(2)
    ropts.SetPageMargins(10)
    report = oedepict.OEReport(ropts)
    cellwidth, cellheight = report.GetCellWidth(), report.GetCellHeight()
    opts = oedepict.OE2DMolDisplayOptions(cellwidth, cellheight, oedepict.OEScale_Default * 0.5)
    opts.SetAromaticStyle(oedepict.OEAromaticStyle_Circle)
    pen = oedepict.OEPen(oechem.OEBlack, oechem.OEBlack, oedepict.OEFill_On, 1.0)
    opts.SetDefaultBondPen(pen)
    oedepict.OESetup2DMolDisplayOptions(opts, itf)
    for i, mol in enumerate(oemols):
        cell = report.NewCell()
        mol_copy = oechem.OEMol(mol)
        oedepict.OEPrepareDepiction(mol_copy, False, suppress_h)
        disp = oedepict.OE2DMolDisplay(mol_copy, opts)

        oedepict.OERenderMolecule(cell, disp)

    oedepict.OEWriteReport("846 clusters_pdfs/cluster%s.pdf" % label, report)

# Sort based on the Frequency of reactants

In [ ]:
from collections import Counter


common_alds = {}
common_amines = {}


c = 1
for label in mols_by_cluster:
    
    oemols = [ oechem.OEMol(mol) for mol in mols_by_cluster[label]]
    alds = []
    ams = []
    
    for i, mol in enumerate(oemols): #i is the counter for each molecule
        ald = oechem.OEGetSDData(mol, "Aldehyde Reactant")
        am = oechem.OEGetSDData(mol, "Amine Reactant")
        alds.append(ald)
        if not ald in common_alds.keys(): #if an entry for the aldehyde reactant does not exist, then add one and give it a value of 1
            common_alds[ald] = 1
        elif common_alds[ald] != c: #if it does exist and hasn't been added within this cluster already(the maximum the value should be is the count of the cluster), then add one to the count
            common_alds[ald] = common_alds[ald] + 1
            
        ams.append(am)
        if not am in common_amines.keys(): #if an entry for the aldehyde reactant does not exist, then add one and assign in the value of the
            common_amines[am] = 1
        elif common_amines[am] != c:
            common_amines[am] = common_amines[am] + 1   
            
    c+=1

    print("\nCluster No." + str(label))
    print ("\nAldehydes")
    print (Counter(alds))
    print ("\nSecondary Amines")
    print(Counter(ams))
        
# get frequency of reactants appearing between clusters    
print ("\n Aldehyde Appearance in clusters:")
print (common_alds)
print("\n Amine Appearance in clusters:")
print (common_amines)
